# Steam Profile Opera - Creating Stories/Reports from Steam Usage

First step is to install the required libraries and make sure we do not have any dependency issue before we start.

In [ ]:
# Import Block
import datetime as dt
import time
import json
from typing import List, Dict, Optional
import os
import math

import requests
import pandas as pd
import numpy as np
from tinydb import TinyDB, Query
from tinydb.storages import JSONStorage
from tinydb_serialization import SerializationMiddleware
from tinydb_serialization.serializers import DateTimeSerializer
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as offline
from dotenv import load_dotenv

# sets time values
current_time = dt.datetime.now()
two_weeks_ago = current_time - dt.timedelta(weeks=2)
year_month_str = f"{current_time.year}-{current_time.month}"
print(f"Year-Month = {year_month_str}")

# Starts DB
serialization = SerializationMiddleware(JSONStorage)
serialization.register_serializer(DateTimeSerializer(), 'TinyDate')
db = TinyDB('db.json', storage=serialization)
users_tb = db.table('users', cache_size=200)
games_tb = db.table('games', cache_size=400)
gameplay_tb = db.table('gameplay', cache_size=1000)
friend_list_tb = db.table('friends', cache_size=200)

offline.init_notebook_mode()


## Retrieving Constants

This will fetch the information from your `.env` file.

In [ ]:
# Retrieving Constants.
load_dotenv()
STEAM_KEY= os.environ.get("STEAM_KEY")
PLAYER_ID= os.environ.get("PLAYER_ID")
RUN_FRIENDS_STATS = os.environ.get("RUN_FRIENDS_STATS") == "True"
CLEAN_USER_DB= os.environ.get("CLEAN_USER_DB") == "True"
CLEAN_GAMEPLAY_DB= os.environ.get("CLEAN_GAMEPLAY_DB") == "True"

In [ ]:
def get_player_data(player_ids:str, year_month:str=None):
    """
    Retrives user data from DB and if not available will use the Steam API.
    """
    id_list = player_ids.split(",")
    missing_steam_ids = []
    result = []

    """
    for steam_id in id_list:
        if year_month:
            user_query = Query()
            user = users_tb.search((user_query.steamid == steam_id)&(user_query.year_month == year_month))
            if len(user) > 0:
                # print(f"User Found {user[0]}")
                result.append(user[0])
        if not year_month or not user:
            missing_steam_ids.append(steam_id)
    """

    if year_month:
        user_query = Query()
        user_list = users_tb.search((user_query.steamid.one_of(id_list))&(user_query.year_month == year_month))
        retrieved_id_list = [user["steamid"] for user in user_list]
        missing_steam_ids += [player_id for player_id in id_list if player_id not in retrieved_id_list ]
        result += user_list
    
    print(f"Retrieved {len(result)} users from DB.")
    print(f"{len(missing_steam_ids)} users will be fetched from the API.")
    if missing_steam_ids:
        print(f"Fetching player data for {len(missing_steam_ids)} players.")
        missing_steam_id_str = ",".join(missing_steam_ids)
        player_url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={STEAM_KEY}&steamids={player_ids}"
        r = requests.get(player_url)
        player_list = r.json()["response"]["players"]
        
        for player in player_list:
            player["year_month"] = year_month_str
        users_tb.insert_multiple(player_list)
        result += player_list
    print(f"Retrieved {len(result)} user(s) succesfully.")
    return result


    

## Test to Retrieve Player Data

Now, we will fetch information for the given `steam_player_id`. Check out if the `personaname` matches your profile!

In [ ]:
# Retrieves current user data
player_data = get_player_data(PLAYER_ID, year_month_str)[0]

In [ ]:
def get_friend_list(player_id, year_month:str=None)->List[str]:
    """
    Returns the list of steam ids for a given player id.
    Caches the friend list for 2 weeks.
    """
    existing_friend_list = []
    
    if year_month:
        print("Attempting to retrieve friend list from DB...")
        friend_query = Query()
        existing_friend_list_query = friend_list_tb.search((friend_query.steam_id == player_id) & (friend_query.year_month == year_month))
        existing_friend_list = existing_friend_list_query[0] if len(existing_friend_list_query) > 0 else None
        if existing_friend_list:
            print(f"Found a record in DB with size {len(existing_friend_list)}")
    if not existing_friend_list:
        print("Friend List not found in the DB, using the API.")
        friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={STEAM_KEY}&steamid={player_id}&relationship=friend"
        r = requests.get(friend_list_url)
        friend_list_response = r.json()
        friend_list_element = friend_list_response.get("friendslist")
        if not friend_list_element:
            print(f"No friend list information was retrieved - Response =>  {friend_list_response}")
            friend_ids = None
        else:
            friend_list_from_response = friend_list_element["friends"]
            friend_ids = [ item["steamid"] for item in friend_list_from_response]
    
        # saves to DB
        friend_list_tb.insert({
            "steam_id": player_id,
            "friend_ids":friend_ids,
            "year_month": year_month_str
        })

        return friend_ids
    return existing_friend_list["friend_ids"]



## Friend List Information

Now we will fetch information about all your friends and save it in a Dataframe.

In [ ]:
friend_list_ids = get_friend_list(PLAYER_ID, year_month_str)
if friend_list_ids:
    friends_data = get_player_data(",".join(friend_list_ids), year_month_str)
    friends_df = pd.DataFrame.from_dict(friends_data + [player_data])
    friends_df["last_year_online"] = friends_df["lastlogoff"].apply(lambda x: dt.datetime.fromtimestamp(x).year if not math.isnan(x) else 0)
    friends_df["loccountrycode"] = friends_df["loccountrycode"].fillna("Not Informed")
    FRIEND_LIST_AVAILABLE = True
else:
    friends_df = pd.DataFrame()
    FRIEND_LIST_AVAILABLE = False
    print("Friend List is not available for this steam id.")

### Friend Count per Country

If your friends provided information about their country, it will be grouped and counted.

In [ ]:
# Count Friends by Country
if FRIEND_LIST_AVAILABLE:
    fig = px.histogram(friends_df, x="loccountrycode", color="loccountrycode",
                        labels={
                            "loccountrycode":"Country Code"
                        },
                        title="Friends per Country",
                        color_discrete_sequence=px.colors.qualitative.Pastel,
                        width=640,
                        height=480)
    fig.show()

### Last Time Seen

This graph the year your friends were last seen online.
Maybe you should call and check on some of them? =D

In [ ]:
# Last Seen
if FRIEND_LIST_AVAILABLE:
    fig = px.histogram(friends_df, x="last_year_online", color="last_year_online",
            labels={
                "last_year_online":"Year"
            },
            title="Year Last Seen",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=640,
            height=480)
    fig.update_layout(bargap=0.2)
    fig.show()

In [ ]:
def fetch_gameplay_info(player_ids:str, year_month:str=None):
    """
    Fetches the list of games and played time for a player id.
    """
    id_list = player_ids.split(",")
    missing_steam_ids = []
    result = []
    if year_month:
        gameplay_query = Query()
        gameplay_list = gameplay_tb.search((gameplay_query.player_id.one_of(id_list))&(gameplay_query.year_month == year_month))
        retrieved_id_list = [gameplay["player_id"] for gameplay in gameplay_list]
        missing_steam_ids += [player_id for player_id in id_list if player_id not in retrieved_id_list ]
        result += gameplay_list
    else:
        missing_steam_ids = id_list.copy()
    print(f"Found {len(result)} gameplay entries in DB.")
    print(f"Fetching {len(missing_steam_ids)} entries from API.")
    if missing_steam_ids:
        for idx,player_id in enumerate(id_list):
            if idx%50 == 0 :
                print(f"Current steam_id number - {idx}")
            player_game_list_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={STEAM_KEY}&steamid={player_id}&format=json"
            r = requests.get(player_game_list_url)
            player_game_list_response = r.json()
            response_content = player_game_list_response["response"]
            gameplay = response_content["games"] if response_content else []
            for entry in gameplay:
                entry.update({
                    "player_id":player_id,
                    "year_month":year_month_str
                })
            if year_month:
                gameplay_tb.insert_multiple(gameplay)
            result += gameplay

    return result



In [ ]:
def get_game_details(appids:List[int]):
    """
    Retrieves game details from the local DB or from Steam.
    """
    game_query = Query()
    result = []
    missing_game_details = []
    existing_game_details = games_tb.search(game_query.appid.one_of(appids))
    if existing_game_details:
        for game_details in existing_game_details:
            if game_details.get("success") == True:
                result.append(game_details)
            elif game_details.get("success") is None:
                games_tb.update({"success":True},(game_query.appid == game_details["appid"]))
        found_game_detail_ids = [ game_detail["appid"] for game_detail in existing_game_details ]
        missing_game_details += [ game_id for game_id in found_game_detail_ids if game_id not in appids]
    else:
        missing_game_details += appids
        
    if missing_game_details:
        for missing_game_id in missing_game_details:
            print(f"Game Details for {str(appid)} NOT found in DB.")
            time.sleep(0.7)  # had to add this because the request were being blocked.
            game_details_url = f"http://store.steampowered.com/api/appdetails?appids={appid}"
            r = requests.get(game_details_url)
            response_json = r.json()
            # print(response_json)
            if (response_json[str(appid)]['success'] == 'true' or response_json[str(appid)]['success'] == True):
                print(f"Saving to DB.")
                game_data = response_json[str(appid)]['data']
                game_data.update({
                    "appid":appid,
                    "success":True})
                games_tb.insert(game_data)
                result.append(game_data)
            else:
                print(f"Saving failed record to DB - {response_json}")
                games_tb.insert({
                    "success":False,
                    "appid":appid,
                })
    return result

    

In [ ]:
def build_df_for_users(player_ids:str, who_is:str=None,year_month:str=None ):

    def extract_release_year(date_dic:dict):
        if type(date_dic)==dict and date_dic["date"]:
            result_date = None
            try:
                result_date = dt.datetime.strptime(date_dic["date"],"%d %b, %Y")
            except ValueError:
                try:
                    result_date = dt.datetime.strptime(date_dic["date"],"%b %d, %Y")
                except ValueError:
                    try:
                        result_date = dt.datetime.strptime(date_dic["date"],"%d %b %Y")
                    except ValueError:
                        return None
            return int(result_date.year)

    
    all_gameplay_list = fetch_gameplay_info(player_ids,year_month)
    print(f"Found {len(all_gameplay_list)} gameplay entries.")
    
    
    if not all_gameplay_list:
        return pd.DataFrame()

    game_id_list = list(set([entry["appid"] for entry in all_gameplay_list]))
    print(f"Total games - {len(game_id_list)}")
    game_details_list = []

    game_data_list = get_game_details(game_id_list)

    for game_data in game_data_list:
  
        game_info = {
            "appid": game_data["appid"],
            "name": game_data["name"],
            "genres": game_data.get("genres"),
            "metacritic":game_data.get("metacritic"),
            "short_description":game_data.get("short_description"),
            "header_image":game_data.get("header_image"),
            "categories":game_data.get("categories"),
            "release_date":game_data.get("release_date"),
            "content_descriptors":game_data.get("content_descriptors"),
            "developers":game_data.get("developers"),
            "publishers":game_data.get("publishers")
        }

        if who_is:
            game_info.update({
            "who_is": who_is
        })
        game_details_list.append(game_info)
            
    print("Done Fetching data.")
    gameplay_df = pd.DataFrame.from_dict(all_gameplay_list)
    game_info_df = pd.DataFrame.from_dict(game_details_list)
    result = pd.merge(gameplay_df,game_info_df, how="left",left_on="appid",right_on="appid")


    result["metacritic_score"] = result["metacritic"].apply(lambda x: x["score"] if type(x)==dict else None)
    result["release_year"] = result["release_date"].apply(extract_release_year)
    result["release_year"] = result["release_year"].fillna(0.0).astype(int)
    result["release_year"].replace(0, np.nan, inplace=True)
    result["pc_developer"] = result["developers"].apply(lambda x: x[0] if type(x)==list else None)
    result["genres_list"] = result["genres"].apply(lambda x: [item["description"] for item in x] if type(x)==list else None)

    return result

        

## Gameplay Information

Now we will start fetching information about your games and gameplay.
First we see what is the release year of your games.

In [ ]:
gameplay_df = build_df_for_users(PLAYER_ID, year_month=year_month_str, who_is="me") 
# gameplay_df = build_df_for_user(76561198021990176)

In [ ]:
exploded_genre_list = gameplay_df[["name","genres_list","playtime_forever"]].explode("genres_list")
exploded_genre_list["playtime_forever"] = exploded_genre_list["playtime_forever"].apply(lambda x: x//60)
exploded_genre_list.reset_index(inplace=True)
top_10_count_by_genre = exploded_genre_list.groupby(['genres_list'])\
    .agg({'genres_list':'count'})\
    .rename(columns={'genres_list':'count_genres_list'})\
    .sort_values('count_genres_list', ascending=False)\
    .head(10).copy()

In [ ]:
top_10_count_by_genre = top_10_count_by_genre.reset_index()
fig = px.bar(top_10_count_by_genre, x='count_genres_list', y='genres_list', color="genres_list",
            labels={
                "count_genres_list":"Count of Games",
                "genres_list":"Genre"
            },
            title="Count of Games by Genre",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=640,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
top_10_playtime_by_genre = exploded_genre_list.groupby(['genres_list'])\
    .agg({'playtime_forever':'sum'})\
    .rename(columns={'playtime_forever':'sum_playtime_forever'})\
    .sort_values('sum_playtime_forever', ascending=False)\
    .head(10).copy()
top_10_playtime_by_genre = top_10_playtime_by_genre.reset_index()
fig = px.bar(top_10_playtime_by_genre, x='sum_playtime_forever', y='genres_list', color="genres_list",
            labels={
                "sum_playtime_forever":"Playtime in hours",
                "genres_list":"Genre"
            },
            title="Total Playtime by Genre",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=640,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Count By Release Year
fig = px.histogram(gameplay_df[gameplay_df["release_year"]>0], x="release_year", color="release_year",
        labels={
            "release_year":"Year"
        },
        title="Games by Year Released",
        color_discrete_sequence=px.colors.qualitative.Pastel,
        width=640,
        height=480)
fig.update_layout(showlegend=False)
fig.update_layout(bargap=0.2)
fig.show()

### My Top 10 Games by Play Time
The next graph will list the top 10 games by playtime in hours.

In [ ]:
# My top 10 Games by Play Time

my_top_10 = gameplay_df.sort_values("playtime_forever", ascending=False).head(10).copy()
my_top_10["playtime_forever"] = my_top_10["playtime_forever"].apply(lambda x: x//60)
fig = px.bar(my_top_10, x='playtime_forever', y='name', color="name",
            labels={
                "playtime_forever":"Playtime in Hours",
                "name":"Game Name"
            },
            title="Top 10 Most Played Games",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=960,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

### Gameplay Time by Release Year and Metacritic

This next graph show how many minutes you spent on each game divided by the release year, and the color shows the metacritic score.
This way you can see where you spent your precious time!

In [ ]:

fig = px.scatter(gameplay_df[(gameplay_df["playtime_forever"]>30)&(gameplay_df["metacritic_score"].notnull())], x="release_year", y="playtime_forever",
                 color="metacritic_score",
                 hover_data=['metacritic_score', 'release_year', 'playtime_forever','name'],
                labels={
                "name": "Game",
                "release_year":"Release Year",
                "playtime_forever":"Play Time in Minutes",
                "metacritic_score":"Metacritic"
                },
                title="Playtime by Game Release Year and Metacritic Score",
                color_continuous_scale="bluered",
                width=640,
                height=480)
fig.show()


### Count By Metacritic Score
Now we will count how many games you have for each Metacritic score.

In [ ]:
# Count by Metacritic
fig = px.histogram(gameplay_df, x="metacritic_score", color="metacritic_score",
        labels={
            "metacritic_score":"Metacritic Score"
        },
        title="Game Count by Metacritic Score",
        color_discrete_sequence=px.colors.qualitative.Pastel,
        width=640,
        height=480)
fig.update_layout(showlegend=False)
fig.update_layout(bargap=0.2)
fig.show()

### Top 10 Game Count per Developer
This Graph will show the top 10 game count per developer in your Steam account.

In [ ]:
# Top 10 count by developer
count_by_developer_df = gameplay_df.groupby(['pc_developer'])\
    .agg({'pc_developer':'count'})\
    .rename(columns={'pc_developer':'count_pc_developer'})\
    .sort_values('count_pc_developer', ascending=False)\
    .head(10)

In [ ]:
fig = px.bar(count_by_developer_df.reset_index(), x='count_pc_developer', y='pc_developer', color="pc_developer",
            labels={
                "count_pc_developer":"Count of Games",
                "pc_developer":"Developer"
            },
            title="Count of Games per Developer",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=640,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

### Playtime by Developer

This next graph will show the top 10 list of how many hours you spent playing titles from each game developer.

In [ ]:
# Top 10 playtime by developer
count_by_developer_df = gameplay_df.groupby(['pc_developer'])\
    .agg({"playtime_forever":"sum"})\
    .rename(columns={'playtime_forever':'sum_playtime_forever'})\
    .sort_values('sum_playtime_forever', ascending=False)\
    .head(10)
count_by_developer_df["sum_playtime_forever"] = count_by_developer_df["sum_playtime_forever"].apply(lambda x: x//60)

In [ ]:
fig = px.bar(count_by_developer_df.reset_index(), x='sum_playtime_forever', y='pc_developer', color="pc_developer",
            labels={
                "sum_playtime_forever":"Playtime in Hours",
                "pc_developer":"Developer"
            },
            title="Playtime by Game Developer",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=640,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

### Games you own but never play by Metacritic

This next graph lists the 10 games you own, ordered by metacritic score, but never played enough - less than 30 minutes to be more precise.
Maybe you can pick one of them in your next free time to explore?

In [ ]:
# Top 10 best rated in metacritid with less than 30 min
unplayed_games_df = gameplay_df[gameplay_df["playtime_forever"]<30]\
    .sort_values('metacritic_score', ascending=False)\
    .head(10)

In [ ]:
fig = px.bar(unplayed_games_df.reset_index(), x='metacritic_score', y='name', color="playtime_forever",
            labels={
                "name":"Game",
                "metacritic_score":"Metacritic Score"
            },
            title="Never Played Games by Metacritic",
            color_discrete_sequence=px.colors.qualitative.Pastel,
            width=960,
            height=480)
fig.update_layout(showlegend=False)
fig.show()

## Friends Gameplay Information

This section will compare your progress in your Steam account in the current date with your friends.
This can be a costly operation, since if the game information is not available in the DB, it will be fetched from Steam using the API, but this must be done one by one and there is a request limit, so it has a sleep function to avoid trespassing this limit.
This can be enabled/disabled in the env file.



In [ ]:
RUN_FRIENDS_STATS=True

In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    all_gameplay_df = gameplay_df.copy()
    friend_gameplay_df = build_df_for_users(",".join(friend_list_ids), who_is="friend")
    all_gameplay_df = pd.concat([all_gameplay_df,friend_gameplay_df])

### My Top 5 Games by Playtime vs My Friend List

The next graph to be evaluated is your top 5 games by total playtime in hours, compared with your friend list values.
In this graph you can see if any of your friends is playing more time than you in your top 5.

In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    final_gameplay_players_df = pd.merge(all_gameplay_df,friends_df[["steamid","personaname","realname","loccountrycode","timecreated","last_year_online","avatar" ]], how="left",left_on="player_id",right_on="steamid")
    my_top_5 = gameplay_df.sort_values("playtime_forever", ascending=False).head(5)["name"]
    played_all_gameplay_df = final_gameplay_players_df[final_gameplay_players_df["playtime_forever"] > 60].copy()
    played_all_gameplay_df["playtime_forever"] = played_all_gameplay_df["playtime_forever"].apply(lambda x: x//60)
    


In [ ]:
played_all_gameplay_df.columns

In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    fig = px.strip(played_all_gameplay_df[played_all_gameplay_df["name"].isin(my_top_5)], 
                        x="playtime_forever", y="name", color='who_is',
                       hover_data=[  'playtime_forever','personaname'],
                        labels={
                        "personaname":"Steam Name",
                        "who_is":"Friend",
                        "name":"Game",
                        "playtime_forever":"Playtime in Hours"
                        },
                        title="My Top 5 Games by Playtime",
                        color_discrete_sequence=px.colors.qualitative.Pastel,
                        width=640,
                        height=480)
    fig.show()

### Who played the most so far?

This next graph will show the top 10 of your friends by total playtime (yourself included).


In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    # Merge DFs
    final_gameplay_players_df = pd.merge(all_gameplay_df,friends_df[["steamid","personaname","realname","loccountrycode","timecreated","last_year_online","avatar" ]], how="left",left_on="player_id",right_on="steamid")


In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    top_10_gameplay = final_gameplay_players_df.groupby(['steamid','personaname'], as_index=False)\
        .agg({"playtime_forever":"sum"}, axis="columns")\
        .sort_values(ascending=False, by="playtime_forever")\
        .head(10)
    top_10_gameplay["playtime_forever"] = top_10_gameplay["playtime_forever"].apply(lambda x: x//60)


In [ ]:
if RUN_FRIENDS_STATS and FRIEND_LIST_AVAILABLE:
    fig = px.bar(top_10_gameplay.reset_index(), x='playtime_forever', y='personaname', color="personaname",
                labels={
                    "playtime_forever":"Playtime in Hours",
                    "personaname":"Steam Name"
                },
                title="Who played the most?",
                color_discrete_sequence=px.colors.qualitative.Pastel,
                width=640,
                height=480)
    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
if CLEAN_USER_DB:
    print("Removing User Data")
    users_tb.truncate()
    print(f"Users size is {len(users_tb)}")
    friend_list_tb.truncate()
    print(f"Friend List size is {len(friend_list_tb)}")
if CLEAN_GAMEPLAY_DB:
    print("Removing Gameplay Data")
    gameplay_tb.truncate()
    print(f"Gameplay List size is {len(gameplay_tb)}")
    